# Creating supercells with pymatgen

The [Pymatgen](https://pymatgen.org/) python library allows to setup
solid-state calculations using a flexible set of classes as well as an API
to an online data base of structures. Its `Structure` and `Lattice`
objects are directly supported by the DFTK `load_atoms` and `load_lattice`
functions, such that DFTK may be readily used to run calculation on systems
defined in pymatgen. Using the `pymatgen_structure` function a conversion
from DFTK to pymatgen structures is also possible. In the following we
use this to create a silicon supercell and find its LDA ground state
using direct minimisation.

First we setup the silicon lattice in DFTK.

In [1]:
using DFTK

a = 10.263141334305942  # Lattice constant in Bohr
lattice = a / 2 .* [[0 1 1.]; [1 0 1.]; [1 1 0.]]
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms = [Si => [ones(3)/8, -ones(3)/8]];

Next we make a `[2, 2, 2]` supercell using pymatgen

In [2]:
pystruct = pymatgen_structure(lattice, atoms)
pystruct.make_supercell([2, 2, 2])
lattice = load_lattice(pystruct)
atoms = [Si => [s.frac_coords for s in pystruct.sites]];

Setup an LDA model and discretize using
a single kpoint and a small `Ecut` of 5 Hartree.
Notice that `PlaneWaveBasis` defaults to just using the ``Γ``-point
if no ``k``-point options are provided.

In [3]:
model = model_LDA(lattice, atoms)
basis = PlaneWaveBasis(model, 5)

PlaneWaveBasis (Ecut=5.0, 1 kpoints)

Find the ground state using direct minimisation (always using SCF is boring ...)

In [4]:
scfres = direct_minimization(basis, tol=1e-5);

Iter     Function value   Gradient norm 
     0     1.115453e+02     1.819613e+00
 * time: 0.1235508918762207
     1     1.000567e+01     1.059304e+00
 * time: 0.9173150062561035
     2    -1.136666e+01     1.208372e+00
 * time: 1.049293041229248
     3    -3.377273e+01     9.749955e-01
 * time: 1.2084510326385498
     4    -4.692682e+01     7.267260e-01
 * time: 1.3589880466461182
     5    -5.693742e+01     2.183846e-01
 * time: 1.5037388801574707
     6    -5.994876e+01     1.623136e-01
 * time: 1.6098740100860596
     7    -6.106250e+01     5.000771e-02
 * time: 1.7232630252838135
     8    -6.148213e+01     5.153591e-02
 * time: 1.8322780132293701
     9    -6.167780e+01     2.971900e-02
 * time: 1.9467239379882812
    10    -6.186246e+01     2.926791e-02
 * time: 2.054007053375244
    11    -6.198921e+01     2.651057e-02
 * time: 2.168684959411621
    12    -6.206395e+01     1.865856e-02
 * time: 2.2761659622192383
    13    -6.211491e+01     1.564579e-02
 * time: 2.3889489173889

In [5]:
scfres.energies

Energy breakdown:
    Kinetic             25.7671064
    AtomicLocal         -18.8557674
    AtomicNonlocal      14.8522648
    Ewald               -67.1831486
    PspCorrection       -2.3569765
    Hartree             4.8485367 
    Xc                  -19.3336818

    total               -62.261666462179
